In [ ]:
#El porcentaje de contenidos que están publicados cuya última edición no tiene comentario para los usuarios que realizaron 1, >10 y >100 de las últimas ediciones (⭐⭐)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk+
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 336 kB in 2s (142 kB/s)
Reading package lists... Done
Building dependency tree       
Reading state information... Done
26 packages can be upgraded. Run 'apt list --upgradable' to see them.
open

In [ ]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('/content/drive/MyDrive/Organizacion De Datos/Trabajo_practico/contents.csv', header=True, inferSchema=True)
rdd = df.rdd

/usr/local/lib/python3.9/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
rdd.map(lambda x: (x.revisor_id  , 1) ).reduceByKey(lambda a,b:a+b).map(lambda x: [x[1]==1, x[1]>10 ,x[1]>100] ).reduce(lambda a,b: (a[0]+b[0], a[1]+b[1], a[2]+b[2]))

(24776, 6250, 1822)

In [ ]:
rdd.map(lambda x: (x.revisor_id  , 1) ).countByKey()

In [ ]:
rdd.map(lambda x: (x.revisor_id , (x.revisor_comment==None, 1)) ).reduceByKey(lambda a,b:(a[0]+b[0],a[1]+b[1])).take(10)

[(None, (77544, 193756)),
 ('3615366', (238, 2905)),
 ('6331749', (38, 47)),
 ('15514', (44, 1216)),
 ('1237851', (104, 760)),
 ('6179998', (False, 1)),
 ('24149', (1, 3431)),
 ('6245048', (0, 9524)),
 ('5045504', (0, 417275)),
 ('5862986', (31, 248))]

In [ ]:
rdd.map(lambda x: (x.revisor_id , (x.revisor_comment==None, 1)) ).reduceByKey(lambda a,b:(a[0]+b[0],a[1]+b[1])).map(lambda x:(x,(x[1][0]==1,x[1][0]>10,x[1][0]>100))).take(5)

[((None, (77544, 193756)), (False, True, True)),
 (('3615366', (238, 2905)), (False, True, True)),
 (('6331749', (38, 47)), (False, True, False)),
 (('15514', (44, 1216)), (False, True, False)),
 (('1237851', (104, 760)), (False, True, True))]

In [ ]:
#rdd.map(lambda x: ((x.revisor_id, x.revisor_comment==None) ) , 1 ).reduceByKey(lambda a,b:a+b).take(5)

In [ ]:
#rdd.map(lambda x: ((x.revisor_id, x.revisor_comment==None) ) , 1 ).reduceByKey(lambda a,b:a+b).map(lambda x: (x[1] == 1 , x[1]>10 , x[1]>100)).take(5)

In [ ]:
#rdd.map(lambda x: ((x.revisor_id, x.revisor_comment==None) ) , 1 ).reduceByKey(lambda a,b:a+b).map(lambda x: (x,(x[1] == 1, x[1]>10, x[1]>100)) ).take(5)


In [ ]:
#rdd.map(lambda x: ((x.revisor_id, x.revisor_comment==None) ) , 1 ).reduceByKey(lambda a,b:a+b).map(lambda x: (x[1] == 1, x[1]>10, x[1]>100) ).reduce(lambda a,b: (a[0]+b[0], a[1]+b[1], a[2]+b[2]))

In [ ]:
#rdd.map(lambda x: ((x.revisor_id, x.revisor_comment==None) ) , 1 ).reduceByKey(lambda a,b:a+b).map(lambda x: (('1',x[1] == 1) , ('10',x[1]>10 ), ('100',x[1]>100))).reduceByKey(lambda a,b:a+b).take(5)

In [ ]:
#rdd.map(lambda x: x.revisor_id and x.revisor_comment == None).count()

In [ ]:
#rdd.map(lambda x: ((x.revisor_id, x.revisor_comment==None) ) , 1 ).reduceByKey(lambda a,b:a+b).map(lambda x: ( (1 , x[1] == 1) , (10, x[1]>10) , (100,x[1]>100) )).countByKey()
# grupo 1 = 44567
#.reduceByKey( lambda a,b: a*100 / (a+b) )

In [ ]:
#rdd.map(lambda x: (x.revisor_id , 1) ).reduceByKey(lambda a,b:a+b).take(10)

In [ ]:
#rdd.map(lambda x: (x.revisor_id , 1) ).reduceByKey(lambda a,b:a+b).map(lambda x: (x[1] == 1, x[1]>10, x[1]>100)).take(10)